In [1]:
from datasets import load_dataset

# Load HotpotQA (distractor + fullwiki)
ds_distractor = load_dataset("hotpotqa/hotpot_qa", "distractor")
ds_fullwiki = load_dataset("hotpotqa/hotpot_qa", "fullwiki")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

distractor/train-00000-of-00002.parquet:   0%|          | 0.00/166M [00:00<?, ?B/s]

distractor/train-00001-of-00002.parquet:   0%|          | 0.00/166M [00:00<?, ?B/s]

distractor/validation-00000-of-00001.par(…):   0%|          | 0.00/27.5M [00:00<?, ?B/s]

...

fullwiki/validation-00000-of-00001.parqu(…):   0%|          | 0.00/28.0M [00:00<?, ?B/s]

fullwiki/test-00000-of-00001.parquet:   0%|          | 0.00/27.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7405 [00:00<?, ? examples/s]

In [2]:
# Pre-processing
def normalize_text(text: str) -> str:
    return " ".join(text.strip().split())

def build_context(example):
    # context is a list of [title, [sentences...]]
    ctx = []
    sent_records = []
    for title, sents in example["context"]:
        cleaned_sents = [normalize_text(s) for s in sents]
        joined = " ".join([s for s in cleaned_sents if s])
        if joined:
            ctx.append({"title": title, "text": joined})
        for i, s in enumerate(cleaned_sents):
            if s:
                sent_records.append({"title": title, "sent_id": i, "text": s})
    example["context_flat"] = ctx
    example["sentences_flat"] = sent_records
    example["question_norm"] = normalize_text(example["question"])
    # supporting_facts is list of [title, sentence_id]
    example["supporting_fact_set"] = set(
        (title, int(sent_id)) for title, sent_id in example["supporting_facts"]
    )
    return example

In [5]:
# Choose a split for experiments

# Pre-processing
def normalize_text(text: str) -> str:
    return " ".join(text.strip().split())

def build_context(example):
    # context is a list of [title, sentence1, sentence2, ...]
    ctx = []
    sent_records = []
    for doc in example["context"]:
        title = doc[0]
        sents = doc[1:]
        cleaned_sents = [normalize_text(s) for s in sents]
        joined = " ".join([s for s in cleaned_sents if s])
        if joined:
            ctx.append({"title": title, "text": joined})
        for i, s in enumerate(cleaned_sents):
            if s:
                sent_records.append({"title": title, "sent_id": i, "text": s})
    example["context_flat"] = ctx
    example["sentences_flat"] = sent_records
    example["question_norm"] = normalize_text(example["question"])
    # supporting_facts is list of [title, sentence_id]
    example["supporting_fact_set"] = set(
        (sf[0], int(sf[1])) for sf in example["supporting_facts"] if len(sf) == 2
    )
    return example

train_ds = ds_distractor["train"].map(build_context)
val_ds = ds_distractor["validation"].map(build_context)

print("Train size:", len(train_ds))
print("Val size:", len(val_ds))
print("Example keys:", train_ds.column_names)


Map:   0%|          | 0/90447 [00:00<?, ? examples/s]

Map:   0%|          | 0/7405 [00:00<?, ? examples/s]

Train size: 90447
Val size: 7405
Example keys: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context', 'context_flat', 'sentences_flat', 'question_norm', 'supporting_fact_set']
